In [ ]:
# Import the libraries

import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import measure, io, img_as_ubyte
from skimage.color import rgb2gray, rgba2rgb
from skimage.filters import threshold_otsu

In [ ]:
def gray_img(img):
    """
    Takes an image as the argument, returns it in grey scale.
    
    Parameters
    ----------
    img: The western blot image in the directory
    
    Returns
    ----------
    Same image in grey scale
    
    """
    
    if img.split(".")[-1] == "jpg":
        return img_as_ubyte(rgb2gray(io.imread(img)))
    
    else:
        return img_as_ubyte(rgb2gray(rgba2rgb(io.imread(img))))

In [ ]:
def thresh_img(grey_img):
    """
    Takes an grey scale image, returns threshold image with Otsu algorithm.
    
    Parameters
    ----------
    grey_img: the grey scale image of western blot provided by gray_img function
    
    Returns
    ----------
    Thresholded image from the grey scale image provided.
    
    """
    
    segmented = grey_img < threshold_otsu(grey_img)
    return segmented

In [ ]:
def band_labeling(original_img, threshold_img):
    """
    Takes an gray scale image, labels all the objects in it.
    
    Parameters
    ----------
    original_img: The raw form of western blot image provided by gray_img function
    threshold_img: Thresholded image provided by thresh_img function
    
    Returns
    ----------
    Thresholded image with all the objects labeled
    """
    
    return measure.label(threshold_img, connectivity = original_img.ndim)

In [ ]:
def band_separation(threshold_img):
    """
    Takes an thresholded image, finds the columns where each object is separated from the other.
    
    Parameters
    ----------
    thresh_img: Thresholded image with all the objects in it labeled
    
    Returns
    ----------
    A tuple of integers
    Every two inegters are for one object. The image is read from left to right. 
    The first integer is the first pixel of the object detected and the second one
    is the last pixel of the object detected.
    
    """
    
    col_img = np.any(threshold_img, axis = 0)
    separated_obj = np.flatnonzero(np.diff(col_img))
    return separated_obj

In [ ]:
def band_measurement(original_img, labeled_img, band_location, conc):
    """
    Calculates the are of each object in image from left to right,
    and saves it into a dictionary with corressponding does of the protein.
    
    Parameters
    ----------
    original_img: The raw western blot image provided from gray_img function
    labeled_img: Thresholded image with labeled object provided from band_labeling function
    band_location: The tuple of integers provided from band_separation function
    conc: List of strings, for all the concentrations used in the wb. Since objects are
    read from left to right, the list's order should be from left to right too.
    
    Returns
    ---------
    A dictionary whose keys are the concentrations and values are the are of the band.
    """
    measurements = {}
    start = 0
    end = 1
    
    for num in range(len(band_location) // 2):
        properties = measure.regionprops(labeled_img[:, band_location[start]: band_location[end]], 
                                         original_img[:, band_location[start]: band_location[end]])
        
        if len(properties) == 0:
            raise Exception("No object detected! Check the image.")
            
        elif len(properties) == 1:
            for ar in properties:
                measurements[conc[num]] = ar.area
                
        else:
            final = max([ar.area for ar in properties])
            measurements[conc[num]] = final
            
        start += 2
        end += 2
       
    return measurements